In [18]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import os
import geopandas as gpd
from xgboost import XGBClassifier,XGBRegressor
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures
from statsmodels.distributions.empirical_distribution import ECDF
import datetime as dt
import pickle
import requests
from collections import defaultdict
import json
from joblib import dump, load

# 서울시 구 판별
## 그 후에 필요한 데이터 불러오기

In [21]:
def goo(data):
    temp=defaultdict(list)
    for i,j in data.items():
        temp[i].append(j)
    data=df(temp)
    data=gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon,data.lat))
    시군구=gpd.read_file('./시군구지도데이터/서울시_경계.shp',encoding='utf-8') # 데이터 업로드 할 때 주소 변경 要
    answer=시군구[시군구['geometry'].apply(lambda i : i.contains(data.geometry[0]))].index
    try:
        return answer[0]
    except:
        return None

# 필요 데이터 불러오기

## 주어진 데이터에서 월,일,요일, 시각, 공휴일 여부, 격자 추출

In [33]:
# 강남구
def extract_feature(data,holiday=holiday):
    is_holiday=0
    temp=defaultdict(list)
    for i,j in data.items():
        temp[i].append(j)
    
    data=df(temp)
    
    data=gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon,data.lat))
    data['시간']=pd.to_datetime(data['current_time'])
    data['요일']=data['시간'].apply(lambda i : i.weekday())
    data['시각']=data['시간'].apply(lambda i : i.hour)
    
    def Grid_match(data):
        lat_min = 37.455
        lat_max = 37.535
        long_min = 127.01
        long_max = 127.125

        lat_bin = np.arange(lat_min, lat_max, 0.005)
        lat_bin = np.append(lat_bin, lat_max)

        long_bin = np.arange(long_min,long_max, 0.005)
        long_bin = np.append(long_bin, long_max)
        for i in range(139):
            a = grid_match[i][0]
            b = grid_match[i][1]
            if (data.lat[0] >= lat_bin[a]) & (data.lat[0] < lat_bin[a+1]):
                if (data.lon[0] >= long_bin[b]) & (data.lon[0] < long_bin[b+1]):
                    return i
    data['grid_index']=Grid_match(data)
    return data

In [34]:
# 중구
def Extract_feature(data,grid,holiday=holiday):
    is_holiday=0
    temp=defaultdict(list)
    for i,j in data.items():
        temp[i].append(j)
    data=df(temp)
    data=gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon,data.lat))
    data['date']=pd.to_datetime(data['current_time'])
    data['연월시']=(data['date'].apply(lambda i : '%d'%(i.year)+('%d'%(i.month)).zfill(2)+('%d'%(i.day)).zfill(2))).astype(int)
    data['month']=data['date'].apply(lambda i : i.month)
    data['days']=data['date'].apply(lambda i : i.day)
    data['hour']=data['date'].apply(lambda i : i.hour)
    data['요일']=data['date'].apply(lambda i : dt.datetime.weekday(i)) 
    data['공휴일']=((data['요일']==6)|(data['연월시'].apply(lambda i : True if i in holiday else False))).astype(int)
    data['grid_num']=grid[grid['geometry'].apply(lambda i : i.contains(data.geometry[0]))].index[0]
    return data

## 주어진 데이터의 위,경도와 가장 가까운 격자 Top k개와 그 격자와의 거리 계산

In [5]:
def top_k_out(data,k=5):
    distance=grid_gangnam.geometry.distance(data.geometry[0])
    top_k=np.argsort(distance)[:k]
    top_k_distance=distance[np.argsort(distance)[:k]]
    real_top_k=[ i for i in top_k if i not in b.keys()]
    real_top_k_distance = top_k_distance[real_top_k]
    not_real_top_k=[ i for i in top_k if i in b.keys() ]
    not_real_top_k_distance = top_k_distance[not_real_top_k]
    return real_top_k, real_top_k_distance

In [161]:
def Top_k_out(data,grid,k=5):
    distance=grid.geometry.centroid.distance(data.geometry[0])
    top_k=np.argsort(distance)[:k]
    top_k_distance=distance[np.argsort(distance)[:k]]
    real_top_k=[i for i in top_k if i not in [2,18]]
    real_top_k_distance = top_k_distance[real_top_k]
    return real_top_k, real_top_k_distance

## 주어진 값에 Softmax 함수를 취하는 method

In [35]:
def softmax(x):
    s=(np.exp(x)).sum()
    return np.exp(x)/s

## 단속될 확률 반환
데이터가 주어졌을 때,  
1) 단속이 잘 안되는 격자일 경우 0 반환  
2) 단속이 되는 격자일 경우  
 2-1) 해당 데이터가 단속이 될 것 인 지 아닐 것 인지를 예측  
    2-1-1) 만약 단속될 확률이 70%미만 일 경우->단속될 확률 반환  
    2-1-2) 만약 단속될 확률이 70%이상 일 경우->3단계로  
     3) 단속이 몇 건으로 될 것 인 지를 예측  
     4) 나오게 된 결과를 확률 값으로 반환  

In [22]:
def caught_or_not(data):
    '''
    단속될 확률 반환
    '''
    if data.grid_index[0] is None: 
        return 0
    else:
        
        processed_data=data[['grid_index','시각','요일']].values
        processed_data=enc.transform(processed_data)
        # 해당 데이터가 단속될 것인지 아닌지 를 분류
        # model load
        
        # predict
        # 단속될 확률이 30% 미만인 경우 
        wasted_prob=xgb_1.predict_proba(processed_data)[:,1]
        if wasted_prob[0]<0.3:
            return wasted_prob[0]
        else:
        # 단속될 확률이 30% 이상인 경우
        # 해당 데이터가 어느 정도로 단속될 지를 예측
        # 해당 데이터가 어느 그리드들과 인접한 지를 계산(5개)
            real_top_k,top_k_distance=top_k_out(data)
            new_data=[[i,data.시각[0],data.요일[0]] for i in real_top_k] # i는 격자
            new_data=enc.transform(new_data)
            # 단속이 된다면 어느정도로 단속될 것 인 지를 예측
            
            nums=xgb_2.predict(new_data)
            
            # 확률로 변환
            probs=ecdf(nums)
            answer = (probs * softmax(top_k_distance)).sum()
            return answer

In [23]:
def Caught_or_not(data,grid,alpha=0.3,k=5):
    '''
    단속될 확률 반환
    '''
    if data.grid_num[0] in [2,18]: 
        return 0
    else:
        # 데이터 전처리
        X=data[['month','days','hour','grid_num','요일','공휴일']]
        # 차원을 2차원으로 확대
        poly=PolynomialFeatures(degree=2)
        X=poly.fit_transform(X)
        # one hot encoder로 변환
        processed_data=one.transform(X)
        
        # 단속될 확률이 30% 미만일 경우
        wasted_prob=xgb_1.predict_proba(processed_data)[:,1]
        if wasted_prob[0]<alpha:
            return wasted_prob[0]
        else:
        # 단속될 확률이 30% 이상일 경우
        # 해당 데이터가 어느 정도로 단속될 지를 예측
        # 해당 데이터가 어느 그리드들과 인접한 지를 계산(k개)
            real_top_k,top_k_distance=Top_k_out(data,grid,k)
            new_data=[]
            for i in real_top_k:
                X['grid_num']=i
                new=X.values.tolist()
                new_data.append(new)
            new_data=poly.fit_transform(new_data)
            # one hot encoder로 변환
            new_data=one.transform(new_data)
            
            # 단속이 된다면 어느정도로 단속될 것 인 지를 예측
            nums=xgb_2.predict(new_data)
            
            # 확률로 변환
            probs=ecdf(nums)
            answer = (probs * softmax(top_k_distance)).sum()
            return answer

## 예시 강남구

In [27]:
if __name__ == "__main__":
    # data input
    dict_string='{"id":1, "lat": 37.500580, "lon": 127.036442, "해당시간": "2020-08-15 00:00"}' # 강남구
    data=json.loads(dict_string)
    
    # 필요 데이터 불러오기
    # 공휴일 데이터a
    holiday=[]
    today=dt.datetime.today()
    for year in range(today.year,today.year+2):
        for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
            URL = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?ServiceKey=SHKN0mFGJKABDNHU9MEyR3vq4%2BE5jaHvVaP8eUt2JgOX1tIWGtc3JvTbwr4a2MRDYa6ONNE0tb7ADSabcrBT7w%3D%3D&_type=json&solYear={year}&solMonth={month}".format(year=year,month=month)
            html = requests.get(URL)
            if html.json()['response']['body']['items']:
                try:
                    holiday.append(html.json()['response']['body']['items']['item']['locdate'])
                except:
                    for i in html.json()['response']['body']['items']['item']:
                        holiday.append(i['locdate'])           
    
    # Grid 데이터(강남구는 없다)
    if goo(data) is not None:
        if goo(data)!=22:
            # 단속 여부(0,1) 분류기 불러오기
            grid=gpd.read_file('./%d.shp'%goo(data)) 
            xgb_1=XGBClassifier()
            xgb_1.load_model('./model1_%d.model'%goo(data)) 

            # 단속이 된다면, 몇 건으로 단속될 지를 판단하는 분류기 불러오기
            xgb_2=XGBRegressor()
            xgb_2.load_model('./model2_%d.model'%goo(data)) 
    
    else:
        print(0)
    # feature 변환자 불러오기
    one = load('./one.joblib') #'./data # 얘는 모든 모델에 대해선 동일할 것임. (강남 제외)
    
    # 중구 일 경우
    if goo(data)==1:
        data2=Extract_feature(data,grid,holiday)
        output=Caught_or_not(data2,grid)
        print(output)
        
    # 강남 구 일 경우
    if goo(data)==22:
        # 강남구에 필요한 데이터 불러오기
        # 1. 강남구 격자 위경도 데이터 불러오기
        with open('./grid_coordi', 'rb') as f: # './data
            a = pickle.load(f)
        with open('./grid_to_zero_coordi', 'rb') as f: #'./data
            b = pickle.load(f) 
        grid_gangnam1=df(a).T
        grid_gangnam2=df(b).T
        grid_gangnam=pd.concat([grid_gangnam1,grid_gangnam2])
        grid_gangnam=gpd.GeoDataFrame(grid_gangnam,geometry=gpd.points_from_xy(grid_gangnam[1],grid_gangnam[0]))

        # 2. 격자 매치 데이터 불러오기
        grid_match=np.load('./grid_match',allow_pickle=True) #'./data

        # 3. 강남구 0,1 분류기 불러오기
        xgb_1=XGBClassifier()
        xgb_1.load_model('./model1.model'%goo(data)) 

        # 4. 단속이 된다면, 몇 건으로 단속될 지를 판단하는 분류기 불러오기
        xgb_2=XGBRegressor()
        xgb_2.load_model('./model2.model'%goo(data)) 

        ## 5. 건수를 확률로 변환하는 변환기 불러오기
        y_train=np.load('./empirical_num.npy') #'./data
        ecdf=ECDF(y_train)

        ## 6. Category Index를 One hot encoder로 변환하는 변환자 불러오기

        enc = load('./enc.joblib') #'./data 
        data2=extract_feature(data,holiday)
        output=caught_or_not(data2)
        print(output)

0.2868507


In [35]:
if __name__ == "__main__":
    # data input
    dict_string='{"id": 1,"lat": 37.551987, "lon": 127.007976, "current_time": "2020-08-16 00:00"}' # 중구
    data=json.loads(dict_string)
    
    # 필요 데이터 불러오기
    # 공휴일 데이터
    holiday=[]
    today=dt.datetime.today()
    for year in range(today.year,today.year+2):
        for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
            URL = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?ServiceKey=SHKN0mFGJKABDNHU9MEyR3vq4%2BE5jaHvVaP8eUt2JgOX1tIWGtc3JvTbwr4a2MRDYa6ONNE0tb7ADSabcrBT7w%3D%3D&_type=json&solYear={year}&solMonth={month}".format(year=year,month=month)
            html = requests.get(URL)
            if html.json()['response']['body']['items']:
                try:
                    holiday.append(html.json()['response']['body']['items']['item']['locdate'])
                except:
                    for i in html.json()['response']['body']['items']['item']:
                        holiday.append(i['locdate'])           
    
    # Grid 데이터(강남구는 없다)
    if goo(data) is not None:
        if goo(data)!=22:
            # 단속 여부(0,1) 분류기 불러오기
            grid=gpd.read_file('./%d.shp'%goo(data)) 
            xgb_1=XGBClassifier()
            xgb_1.load_model('./model1_%d.model'%goo(data)) 

            # 단속이 된다면, 몇 건으로 단속될 지를 판단하는 분류기 불러오기
            xgb_2=XGBRegressor()
            xgb_2.load_model('./model2_%d.model'%goo(data)) 
    
    else:
        print(0)
    # feature 변환자 불러오기
    one = load('./one.joblib') #'./data # 얘는 모든 모델에 대해선 동일할 것임. (강남 제외)
    
    # 중구 일 경우
    if goo(data)==1:
        data2=Extract_feature(data,grid,holiday)
        output=Caught_or_not(data2,grid)
        print(output)
        
    # 강남 구 일 경우
    if goo(data)==22:
        # 강남구에 필요한 데이터 불러오기
        # 1. 강남구 격자 위경도 데이터 불러오기
        with open('./grid_coordi', 'rb') as f: # './data
            a = pickle.load(f)
        with open('./grid_to_zero_coordi', 'rb') as f: #'./data
            b = pickle.load(f) 
        grid_gangnam1=df(a).T
        grid_gangnam2=df(b).T
        grid_gangnam=pd.concat([grid_gangnam1,grid_gangnam2])
        grid_gangnam=gpd.GeoDataFrame(grid_gangnam,geometry=gpd.points_from_xy(grid_gangnam[1],grid_gangnam[0]))

        # 2. 격자 매치 데이터 불러오기
        grid_match=np.load('./grid_match',allow_pickle=True) #'./data

        # 3. 강남구 0,1 분류기 불러오기
        xgb_1=XGBClassifier()
        xgb_1.load_model('./model1.model'%goo(data)) 

        # 4. 단속이 된다면, 몇 건으로 단속될 지를 판단하는 분류기 불러오기
        xgb_2=XGBRegressor()
        xgb_2.load_model('./model2.model'%goo(data)) 

        ## 5. 건수를 확률로 변환하는 변환기 불러오기
        y_train=np.load('./empirical_num.npy') #'./data
        ecdf=ECDF(y_train)

        ## 6. Category Index를 One hot encoder로 변환하는 변환자 불러오기

        enc = load('./enc.joblib') #'./data 
        data2=extract_feature(data,holiday)
        output=caught_or_not(data2)
        print(output)

0.021242326
